**Installing all necessary packages**

In [1]:
%%capture
!pip install gradio transformers transformers-stream-generator qwen-vl-utils
!pip install torchvision torch huggingface_hub spaces accelerate ipython
!pip install pillow av python-docx python-pptx requests numpy reportlab fpdf hf_xet

In [2]:
import gradio as gr
import spaces
from transformers import (
    Qwen2_5_VLForConditionalGeneration,
    Qwen2VLForConditionalGeneration,
    AutoProcessor,
    TextIteratorStreamer,
    AutoTokenizer,
    AutoModelForVision2Seq,
    AutoModelForImageTextToText)
from qwen_vl_utils import process_vision_info
import torch
from PIL import Image
import os
import uuid
import io
from threading import Thread
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Image as RLImage, Paragraph, Spacer
from reportlab.lib.units import inch
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import re
import docx
from docx.enum.text import WD_ALIGN_PARAGRAPH
from pptx import Presentation

**Run app**

In [3]:
#Model used in the app:
#https://huggingface.co/nanonets/Nanonets-OCR-s
#https://huggingface.co/prithivMLmods/Qwen2-VL-OCR-2B-Instruct
#https://huggingface.co/CohereForAI/aya-vision-8b
#https://huggingface.co/mradermacher/olmOCR-7B-0725-GGUF

# Define model options
MODEL_OPTIONS = {
    "Nanonets": "nanonets/Nanonets-OCR-s",
    "PrithivMLmods" : "prithivMLmods/Qwen2-VL-OCR-2B-Instruct",
    #"Reducto": "reducto/RolmOCR"
}

# Preload models and processors into CUDA
models = {}
processors = {}
for name, model_id in MODEL_OPTIONS.items():
    if name == "Nanonets":
      models[name] = Qwen2_5_VLForConditionalGeneration.from_pretrained(
          model_id,
          trust_remote_code=True,
          torch_dtype=torch.bfloat16
      ).to("cuda").eval()
      processors[name] = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
    elif name == "PrithivMLmods":
      models[name] = Qwen2VLForConditionalGeneration.from_pretrained(
          model_id,
          trust_remote_code=True,
          torch_dtype=torch.bfloat16
      ).to("cuda").eval()
      processors[name] = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
    # else:
    #   models[name] = Qwen2_5_VLForConditionalGeneration.from_pretrained(model_id,
    #       trust_remote_code=True,
    #       torch_dtype=torch.bfloat16
    #   ).to("cuda").eval()
    # processors[name] = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
image_extensions = Image.registered_extensions()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.51G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/408 [00:00<?, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json: 0.00B [00:00, ?B/s]

In [4]:
def identify_and_save_blob(blob_path):
    """Identifies if the blob is an image and saves it."""
    try:
        with open(blob_path, 'rb') as file:
            blob_content = file.read()
            try:
                Image.open(io.BytesIO(blob_content)).verify()  # Check if it's a valid image
                extension = ".png"  # Default to PNG for saving
                media_type = "image"
            except (IOError, SyntaxError):
                raise ValueError("Unsupported media type. Please upload a valid image.")

            filename = f"temp_{uuid.uuid4()}_media{extension}"
            with open(filename, "wb") as f:
                f.write(blob_content)

            return filename, media_type

    except FileNotFoundError:
        raise ValueError(f"The file {blob_path} was not found.")
    except Exception as e:
        raise ValueError(f"An error occurred while processing the file: {e}")

In [5]:
@spaces.GPU
def ocr_inference(model_name, media_input, text_input=None):
    """Handles inference for the selected model."""
    model = models[model_name]
    processor = processors[model_name]

    if isinstance(media_input, str):
        media_path = media_input
        if media_path.endswith(tuple([i for i in image_extensions.keys()])):
            media_type = "image"
        else:
            try:
                media_path, media_type = identify_and_save_blob(media_input)
            except Exception as e:
                raise ValueError("Unsupported media type. Please upload a valid image.")

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": media_type,
                    media_type: media_path
                },
                {"type": "text", "text": text_input},
            ],
        }
    ]

    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, _ = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        padding=True,
        return_tensors="pt",
    ).to("cuda")

    streamer = TextIteratorStreamer(
        processor.tokenizer, skip_prompt=True, skip_special_tokens=True
    )
    generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=1024)

    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    buffer = ""
    for new_text in streamer:
        buffer += new_text
        # Remove <|im_end|> or similar tokens from the output
        buffer = buffer.replace("<|im_end|>", "")
        yield buffer

In [6]:
def format_plain_text(output_text):
    """Formats the output text as plain text without LaTeX delimiters."""
    # Remove LaTeX delimiters and convert to plain text
    plain_text = output_text.replace("\\(", "").replace("\\)", "").replace("\\[", "").replace("\\]", "")
    return plain_text

In [7]:
def generate_pdf(media_path, plain_text, font_size, line_spacing, alignment, image_size):
    """Generates a PDF document."""
    filename = f"output_{uuid.uuid4()}.pdf"
    doc = SimpleDocTemplate(
        filename,
        pagesize=A4,
        rightMargin=inch,
        leftMargin=inch,
        topMargin=inch,
        bottomMargin=inch
    )
    styles = getSampleStyleSheet()
    styles["Normal"].fontSize = int(font_size)
    styles["Normal"].leading = int(font_size) * line_spacing
    styles["Normal"].alignment = {
        "Left": 0,
        "Center": 1,
        "Right": 2,
        "Justified": 4
    }[alignment]

    story = []

    # Add image with size adjustment
    image_sizes = {
        "Small": (200, 200),
        "Medium": (400, 400),
        "Large": (600, 600)
    }
    img = RLImage(media_path, width=image_sizes[image_size][0], height=image_sizes[image_size][1])
    story.append(img)
    story.append(Spacer(1, 12))

    # Add plain text output
    text = Paragraph(plain_text, styles["Normal"])
    story.append(text)

    doc.build(story)
    return filename

In [8]:
def generate_docx(media_path, plain_text, font_size, line_spacing, alignment, image_size):
    """Generates a DOCX document."""
    filename = f"output_{uuid.uuid4()}.docx"
    doc = docx.Document()

    # Add image with size adjustment
    image_sizes = {
        "Small": docx.shared.Inches(2),
        "Medium": docx.shared.Inches(4),
        "Large": docx.shared.Inches(6)
    }
    doc.add_picture(media_path, width=image_sizes[image_size])
    doc.add_paragraph()

    # Add plain text output
    paragraph = doc.add_paragraph()
    paragraph.paragraph_format.line_spacing = line_spacing
    paragraph.paragraph_format.alignment = {
        "Left": WD_ALIGN_PARAGRAPH.LEFT,
        "Center": WD_ALIGN_PARAGRAPH.CENTER,
        "Right": WD_ALIGN_PARAGRAPH.RIGHT,
        "Justified": WD_ALIGN_PARAGRAPH.JUSTIFY
    }[alignment]
    run = paragraph.add_run(plain_text)
    run.font.size = docx.shared.Pt(int(font_size))

    doc.save(filename)
    return filename

In [9]:
def generate_markdown_file(media_path, markdown_text, image_size):
    """Generates a Markdown document."""
    filename = f"output_{uuid.uuid4()}.md"
    image_sizes = {
        "Small": "200",
        "Medium": "400",
        "Large": "600"
    }
    markdown_content = f"""
![Image]({media_path})

$${markdown_text}$$
"""
    with open(filename, "w") as f:
        f.write(markdown_content)
    return filename

In [10]:
def generate_latex(media_path, latex_text, image_size):
    """Generates a LaTeX document."""
    filename = f"output_{uuid.uuid4()}.tex"
    image_sizes = {
        "Small": "0.3\\textwidth",
        "Medium": "0.6\\textwidth",
        "Large": "0.9\\textwidth"
    }
    latex_content = f"""
\\documentclass{{article}}
\\usepackage{{graphicx}}
\\usepackage{{geometry}}
\\geometry{{a4paper, margin=1in}}

\\begin{{document}}

\\begin{{figure}}[h!]
    \\centering
    \\includegraphics[width={image_sizes[image_size]}]{{{media_path}}}
\\end{{figure}}

{latex_text}

\\end{{document}}
"""
    with open(filename, "w") as f:
        f.write(latex_content)
    return filename

In [11]:
def generate_document(media_path, output_text, file_format, font_size, line_spacing, alignment, image_size):
    """Generates a document with the input image and plain text output."""
    plain_text = format_plain_text(output_text)
    if file_format == "pdf":
        return generate_pdf(media_path, plain_text, font_size, line_spacing, alignment, image_size)
    elif file_format == "docx":
        return generate_docx(media_path, plain_text, font_size, line_spacing, alignment, image_size)
    elif file_format == "markdown":
        return generate_markdown_file(media_path, plain_text, image_size)
    elif file_format == "latex":
        return generate_latex(media_path, plain_text, image_size)
    else:
        return output_text

In [12]:
def format_markdown(output_text):
    """Formats the output text as Markdown."""
    # This is a basic implementation. You might need to add more sophisticated Markdown formatting
    # depending on the expected content of the output_text.
    markdown_text = output_text.replace('\\(', '$').replace('\\)', '$').replace('\\[', '$$').replace('\\]', '$$')
    return markdown_text

In [13]:
def format_latex(output_text):
    """Formats the output text as LaTeX."""
    # This is a basic implementation. You might need to add more sophisticated LaTeX formatting
    # depending on the expected content of the output_text.
    latex_text = output_text.replace('\\(', '$').replace('\\)', '$').replace('\\[', '$$').replace('\\]', '$$')
    return latex_text

In [36]:
def update_formatted_output(output_text, output_format):
    if output_format == "Plain Text":
        return format_plain_text(output_text)
    elif output_format == "Markdown":
        return format_markdown(output_text)
    elif output_format == "LaTex":
        return format_latex(output_text)
    else:
        return output_text

In [20]:
def classify_latex(latex_code, category_map):
   found_categories = set()
   for category, patterns in category_map.items():
       for pattern in patterns:
           if re.search(pattern, latex_code):
               found_categories.add(category)

   # Ưu tiên các danh mục cụ thể hơn
   if "statistics" in found_categories:
       return "statistics"
   if "summation" in found_categories:
       return "summation"
   if "integral" in found_categories:
       return "integral"
   if "fractions" in found_categories:
       return "fractions"
   if "differential" in found_categories:
       return "differential"
   if "basic_math" in found_categories:
       return "basic_math"
   if not found_categories:
       return "Other"
   # If none of the specific categories are found, return the first found category or "Other"
   return list(found_categories)[0] if found_categories else "Other"

In [21]:
def process_and_categorize(model_name, media_input, text_input):
    # First, run the OCR inference
    ocr_output = ""
    for text_chunk in ocr_inference(model_name, media_input, text_input):
        ocr_output += text_chunk
    yield ocr_output, None # Update output_text first

    # Define the categories and classification function from the user's input
    categories = {
        #Lý thuyết tập hợp
        "set_theory": [r"\\cup", r"\\cap", r"\\setminus", r"\\emptyset", r"\\subseteq", r"\\supseteq", r"\\in", r"\\notin", r"\\forall", r"\\exists"],
        #Thống kê
        "statistics": [r"\\mu", r"\\sigma", r"\\rho", r"\\beta", r"\\sum", r"\\bar", r"\\hat", r"\\mathbb{E}", r"\\mathrm{Var}", r"\\mathrm{Cov}"],
        #Tích phân
        "integral": [r"\\int", r"\\iint", r"\\iiint", r"\\oint", r"d[a-zA-Z]"],
        #Ký hiệu Hy Lạp
        "greek_symbols": [r"\\[a-zA-Z]+", r"[α-ωΑ-Ω]"], # Catch most Greek symbols
        #Đại số tuyến tính
        "linear_algebra": [r"\\begin\{pmatrix\}", r"\\begin\{bmatrix\}", r"\\det", r"\\mathrm{Tr}", r"\\vec", r"\\cdot"],
        #Logarit
        "logarithms": [r"\\log", r"\\ln"],
        "basic_math": [r"\+", r"-", r"\\*", r"/", r"=", r"<", r">", r"\\le", r"\\ge"],
        #Phân số
        "fractions": [r"\\frac"],
        #Vi phân
        "differential": [r"\\frac\{d", r"\\partial"],
        #Lượng giác
        "trig": [r"\\sin", r"\\cos", r"\\tan", r"\\cot", r"\\sec", r"\\csc", r"\\arcsin", r"\\arccos", r"\\arctan"],
        #Tổng
        "summation": [r"\\sum"],
        #Lũy thừa và căn
        "powers_roots": [r"\^", r"\\sqrt"],
        #Lim: giới hạn giải tích
        "limits": [r"\\lim"],
        #Bất đẳng thức
        "inequalities": [r"<", r">", r"\\le", r"\\ge", r"\\neq"],
        #Hình học
        "geometry": [r"\\angle", r"\\triangle", r"\\vec"],
        #Số phức
        "complex_numbers": [r"i", r"j", r"\\mathbb{C}", r"\\Re", r"\\Im"],
        "Other": []
    }

    result_category = classify_latex(ocr_output, categories)
    yield ocr_output, result_category

In [47]:
# CSS for output styling
css = """
  #output {
    height: 500px;
    overflow: auto;
    border: 1px solid #ccc;
  }
.submit-btn {
    background-color: #cf3434  !important;
    color: white !important;
}
.submit-btn:hover {
    background-color: #ff2323 !important;
}
.download-btn {
    background-color: #35a6d6 !important;
    color: white !important;
}
.download-btn:hover {
    background-color: #22bcff !important;
}
.latex-preview {
    height: 500px;
    overflow: auto;
    border: 1px solid #ccc;
}
"""
# Gradio app setup
with gr.Blocks(css=css, theme="bethecloud/storj_theme") as demo:
    gr.Markdown("# **OCR : All you needed**")

    with gr.Tab(label="Image Input"):
        with gr.Row():
            with gr.Column():
                model_choice = gr.Dropdown(
                    label="Model Selection",
                    choices=list(MODEL_OPTIONS.keys()),
                    value="Nanonets"
                )
                input_media = gr.File(
                    label="Upload Image", type="filepath"
                )
                # Incase LaTex: please input "Identify and format any formulas found in the image using LaTeX."
                text_input = gr.Textbox(label="Question", value="OCR the image precisely.")
                submit_btn = gr.Button(value="Submit", elem_classes="submit-btn")

            with gr.Column():
                output_text = gr.Textbox(label="Output Text", lines=7)

                output_format_radio = gr.Radio(choices=["Plain Text", "Markdown", "LaTex"], label="Select Output Format", value="Plain Text", visible=True)
                formatted_output = gr.Textbox(label="Formatted Output", lines=10)

                # Section to display categorized formulas
                gr.Markdown("### Categorized Formulas")
                categorized_formulas_output = gr.Textbox(label="Categorized Formulas")

        submit_btn.click(
            process_and_categorize, [model_choice, input_media, text_input], [output_text, categorized_formulas_output]
        )

        output_text.change(
            update_formatted_output, [output_text, output_format_radio], [formatted_output]
        )
        output_format_radio.change(
            update_formatted_output, [output_text, output_format_radio], [formatted_output]
        )

        with gr.Accordion("Docx/PDF Settings", open=False):
          with gr.Row():
              with gr.Column():
                  line_spacing = gr.Dropdown(
                      choices=[0.5, 1.0, 1.15, 1.5, 2.0, 2.5, 3.0],
                      value=1.5,
                      label="Line Spacing"
                  )
                  font_size = gr.Dropdown(
                      choices=["8", "10", "12", "14", "16", "18", "20", "22", "24"],
                      value="16",
                      label="Font Size"
                  )
                  alignment = gr.Dropdown(
                      choices=["Left", "Center", "Right", "Justified"],
                      value="Justified",
                      label="Text Alignment"
                  )
                  image_size = gr.Dropdown(
                      choices=["Small", "Medium", "Large"],
                      value="Medium",
                      label="Image Size"
                  )
                  file_format = gr.Radio(["pdf", "docx", "markdown", "latex"], label="File Format", value="pdf")

        get_document_btn = gr.Button(value="Get Document", elem_classes="download-btn")

        get_document_btn.click(
            generate_document, [input_media, output_text, file_format, font_size, line_spacing, alignment, image_size], gr.File(label="Download Document")
        )

    with gr.Tab(label="LaTeX Editor"):
        gr.Markdown("Type your LaTeX code in the editor below and see the live preview.")

        def render_latex(latex_code):
            """A function that returns the input LaTeX code to be rendered."""
            return latex_code

        with gr.Row():
            with gr.Column(scale=1):
                latex_input = gr.Code(
                    label="LaTeX Code Editor",
                    language="latex",
                    value="""
                    # Write your LaTeX here
                    $$ E = mc^2 $$
                    """,
                    lines=15
                )
            with gr.Column(scale=1):
                latex_output = gr.Markdown(
                    label="Rendered Preview",
                    elem_classes=["latex-preview"],
                    # The initial value is set to match the input value
                    value="""
                    # Write your LaTeX here
                    $$ E = mc^2 $$
                    """
                )

        # Connect the input editor to the output preview.
        latex_input.change(fn=render_latex, inputs=latex_input, outputs=latex_output)

    # Add an event listener to update the LaTeX editor when the output format changes
    output_format_radio.change(
        lambda output_text, output_format: format_latex(output_text) if output_format == "LaTex" else "",
        [output_text, output_format_radio],
        [latex_input] # Update the input editor with the generated LaTeX
    )

In [48]:
demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ca6eddc5ebdf221f3d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2245, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1972, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1927, in validate_outputs
    rai

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ca6eddc5ebdf221f3d.gradio.live
